## ここで実施すること

- AIシステムのテスト

#### 環境

```

```

In [1]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)
import torch

#### モデル名の定義

huggingfaceリポジトリに公開されている文章生成AIモデルを適当に選定する。

今回はサイバーエージェント社のモデルを使用する。

[huggingfaceリポジトリ](https://huggingface.co/cyberagent/calm3-22b-chat)

このモデルは、日本語に対応している大型の文章生成AIモデルの中でも話題にされている印象がある。

[記事1](https://qiita.com/isanakamishiro2/items/5dea69a3cb33f8ac51e0)

[記事2](https://highreso.jp/edgehub/machinelearning/cyberagentlm3.html)

In [2]:
model_name = "cyberagent/calm3-22b-chat"

#### モデルのダウンロード

huggingfaceのリポジトリから環境にモデルをダウンロードする。

大きなモデルをダウンロードするときは、ディスクもたくさん用意しないといけないし、時間もかかる。

>
>量子化
>
>パラメータを表現する桁数を削減することで、記憶領域の使用量を削減する。
>


>
>トークナイザー
>
>文章を細切れにするための小さな機械学習モデル。
>
>文章生成AIへクエリを入力する前処理で使用する。
>

In [3]:
%%time

## トークナイザー
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    legacy = False
)

## LLM
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = BitsAndBytesConfig(load_in_4bit = True), ## 量子化して軽量に
    torch_dtype = torch.float32 ## 正規化層のみ32bit扱い(重要なところだけ)
)

## 演算にコンピュータのGPUを利用する
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/35.8k [00:00<?, ?B/s]

model-00001-of-00010.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00002-of-00010.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00003-of-00010.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

...

model-00010-of-00010.safetensors:   0%|          | 0.00/1.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

CPU times: user 8min 9s, sys: 18min 41s, total: 26min 50s
Wall time: 20min 8s


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(65024, 6144, padding_idx=1)
    (layers): ModuleList(
      (0-47): 48 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=6144, out_features=6144, bias=False)
          (k_proj): Linear4bit(in_features=6144, out_features=6144, bias=False)
          (v_proj): Linear4bit(in_features=6144, out_features=6144, bias=False)
          (o_proj): Linear4bit(in_features=6144, out_features=6144, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=6144, out_features=16384, bias=False)
          (up_proj): Linear4bit(in_features=6144, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=16384, out_features=6144, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((6144,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((6144,), eps=1e-05)
      )
    )
    (norm): Lla

#### メッセージを作成する

指定のフォーマットに基づいてAIへの指示を作成する。

今回のモデルは成熟しているので、あらかじめフォーマットが決められている。

[フォーマット](https://huggingface.co/cyberagent/calm3-22b-chat)

In [4]:
messages = [
    {"role": "system", "content": "あなたは親切なAIアシスタントです。"},
    {"role": "user", "content": "AIによって私たちの暮らしはどのように変わりますか？"}
]

#### トークナイズ

作成したメッセージをトークナイズする。

文章は、トークンという意味の断片に分割される。

断片にはIDが充てられている。

input_idsには、トークンIDの配列が格納されている。

In [5]:
%%time

input_ids = tokenizer.apply_chat_template(
    messages, 
    add_generation_prompt = True, 
    return_tensors = "pt"
).to(model.device)

input_ids

CPU times: user 10 ms, sys: 8.94 ms, total: 18.9 ms
Wall time: 13.8 ms


tensor([[65002, 11940,   186, 14052, 32091,   278,  7273, 14010, 16099,   434,
           260, 65001,   186, 65002,  4982,   186,  7273,  1501,  5015, 23045,
           276, 10761, 25978,   306,  1018, 65001,   186, 65002,   578, 15199,
           186]], device='cuda:0')

#### 文章生成

>
>temperature
>
>値が低いほど、創造性が高くなる。
>
>文章生成AIは確率によって次のトークンを選ぶが、尤もらしいトークンを選び続けると「ありがちで」「無難な」「アグレッシブではない」文章が得られがちである。
>
>この値を下げることで、文章のクセ？に揺らぎを与えることができる。


In [7]:
%%time

output_ids = model.generate(
    input_ids,
    max_new_tokens=512, ## 文章の長さに関係
    temperature=0.5     ## 文章の創造性に関係
)

output_ids

OutOfMemoryError: CUDA out of memory. Tried to allocate 144.00 MiB. GPU 0 has a total capacity of 15.72 GiB of which 5.44 MiB is free. Process 2304675 has 15.71 GiB memory in use. Of the allocated memory 15.32 GiB is allocated by PyTorch, and 203.90 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)